In [11]:
# IO
out_fp = "output/"
out_fn = out_fp + "kernel_test"

In [12]:
# MCMC settings
mvi = 1
mni = 1
n_gen = 1e4
sample_freq = floor(n_gen/1e2)

In [13]:
# data dimensions
n_taxa = 10
n_sites = 100

In [14]:
# create dummy taxa
for (i in 1:n_taxa) {
    taxa[i] = taxon("T"+i, 0.0)
}
taxa

   [ T1, T2, T3, T4, T5, T6, T7, T8, T9, T10]


In [15]:
# create unit Yule tree
birth ~ dnExp(1)
phy ~ dnBDP(lambda=birth, mu=0., rootAge=1, taxa=taxa)

# tree moves
mv[mvi++] = mvNNI(phy, weight=n_taxa)
mv[mvi++] = mvFNPR(phy, weight=n_taxa/2)
mv[mvi++] = mvNodeTimeSlideUniform(phy, weight=n_taxa)

# print the tree's value
phy

  
   ((T10[&index=10]:0.732050,T6[&index=9]:0.732050)[&index=11]:0.267950,(((T5[&index=8]:0.887678,T7[&index=7]:0.887678)[&index=12]:0.018420,T8[&index=6]:0.906098)[&index=13]:0.009943,((T1[&index=5]:0.206622,T4[&index=4]:0.206622)[&index=14]:0.575270,(T2[&index=3]:0.660065,(T3[&index=2]:0.445260,T9[&index=1]:0.445260)[&index=15]:0.214805)[&index=16]:0.121827)[&index=17]:0.134150)[&index=18]:0.083959)[&index=19]:0.000000;


In [16]:
# base frequencies
pi ~ dnDirichlet(rep(1,4))
mv[mvi++] = mvSimplexElementScale(pi, alpha=10, weight=4)

# exchangeability rates
er ~ dnDirichlet(rep(1,6))
mv[mvi++] = mvSimplexElementScale(er, alpha=10, weight=6)

# GTR rate matrix
Q := fnGTR(exchangeRates=er,
           baseFrequencies=pi)
    
# print the matrix's value
Q

   [ [ -0.6057, 0.0800, 0.5109, 0.0148 ] ,
     1.1188, -1.4449, 0.0335, 0.2926 ] ,
     1.6462, 0.0077, -1.9370, 0.2830 ] ,
     0.2856, 0.4042, 1.6965, -2.3862 ] ]


In [17]:
# build phylogenetic CTMC
seq ~ dnPhyloCTMC(tree=phy,
                  Q=Q,
                  branchRates=1.,
                  nSites=n_sites,
                  type="DNA")

# treat the simulated data as the likelihood
seq.clamp(seq)

In [18]:
# save some values
writeNexus(seq, filename=out_fn+".sim.nex")
writeNexus(phy, filename=out_fn+".sim.tre")

In [19]:
# create monitors
mn[mni++] = mnScreen(pi, printgen=sample_freq)
mn[mni++] = mnModel(filename=out_fn+".model.log",
                    printgen=sample_freq)
mn[mni++] = mnFile(phy,
                   filename=out_fn+".tre",
                   printgen=sample_freq)

   Error:	End of line while in quote
   Syntax error while reading character ')' at position 40 in command:
                      printgen=sample_freq)


In [20]:
# create MCMC
mdl = model(phy)
ch = mcmc(mdl,mv,mn)
ch.run(n_gen)


   Running MCMC simulation
   This simulation runs 1 independent replicate.
   The simulator uses 5 different moves in a random move schedule with 35 moves per iteration

Iter        |      Posterior   |     Likelihood   |          Prior   |          pi[1]      |       pi[2]      |       pi[3]      |       pi[4]   |    elapsed   |        ETA   |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0           |       -877.092   |       -865.879   |       -11.2134   |       0.697543      |    0.049879      |    0.216467      |   0.0361115   |   00:00:00   |   --:--:--   |
100         |       -873.718   |       -862.509   |       -11.2089   |       0.680425      |   0.0768707      |    0.210185      |   0.0325194   |   00:00:01   |   --:--:--   |
200         |       -874.376   |       -864.049   |       -10.3267   |       0.665053      |   0.0809766      |    0.210